# Make a Choropleth for the Monthly Milk-Report Data
_Paulo G. Martinez,_ 12-7-2020

Where would I get that data from?
- Let's use the data from the tidied up monthly milk report on Google sheets cause that's what Matt is using.

**Imports**

In [1]:
# for tabular manipulation
import pandas as pd
# for OS-agnostic path handling
import pathlib
# for interactive plots
import plotly.express as px
import plotly.graph_objects as go

# for regex
import re

In [2]:
verbose = True

**read the data**

In [3]:
!ls data/

aug-milk-prod-24stRprtMnthsLngYrsPrsntLst.tsv
milk_production_national_month_2019_2020.tsv
milk_production_national_monthly.tsv


In [4]:
# declare the path
data_path = pathlib.Path("data/aug-milk-prod-24stRprtMnthsLngYrsPrsntLst.tsv")
# read the data
df = pd.read_csv(filepath_or_buffer=data_path,
                                  # using tab separated values
                                  sep = "\t",
                                 )

In [5]:
df.head()

,Milk Cows,Milk Per Cow,Milk Production (lbs),State,Year,Month
0,"195,000","1,940","378,000,000",Arizona,2019,July
1,"1,725,000","1,965","3,390,000,000",California,2019,July
2,"187,000","2,205","412,000,000",Colorado,2019,July
3,"115,000","1,670","192,000,000",Florida,2019,July
4,"81,000","1,765","143,000,000",Georgia,2019,July


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Milk Cows              96 non-null     object
 1   Milk Per Cow           96 non-null     object
 2   Milk Production (lbs)  96 non-null     object
 3   State                  96 non-null     object
 4   Year                   96 non-null     int64 
 5   Month                  96 non-null     object
dtypes: int64(1), object(5)
memory usage: 4.6+ KB


## rename the columns to match the presentation

In [7]:
df = df.rename(columns = {"Milk Per Cow":"Milk Per Cow (Lbs)",
                    "Milk Production (lbs)": "Milk Production (Lbs)",
                    })
df.head()

,Milk Cows,Milk Per Cow (Lbs),Milk Production (Lbs),State,Year,Month
0,"195,000","1,940","378,000,000",Arizona,2019,July
1,"1,725,000","1,965","3,390,000,000",California,2019,July
2,"187,000","2,205","412,000,000",Colorado,2019,July
3,"115,000","1,670","192,000,000",Florida,2019,July
4,"81,000","1,765","143,000,000",Georgia,2019,July


## prepare the columns for tidy plotting
### compress Year and Month into a datetime index

In [8]:
# overwrite the index
df.index = pd.to_datetime(
    # unpack month and year values into string expression of date
    [f"{m}-{y}" for m,y in df[["Month", "Year"]].values]
    # offset to the end of the month
) + pd.offsets.MonthEnd(0)

# give the index a helpful name
df.index.name = "Date"

In [9]:
df.head()

,Milk Cows,Milk Per Cow (Lbs),Milk Production (Lbs),State,Year,Month
Date,,,,,,
2019-07-31,"195,000","1,940","378,000,000",Arizona,2019,July
2019-07-31,"1,725,000","1,965","3,390,000,000",California,2019,July
2019-07-31,"187,000","2,205","412,000,000",Colorado,2019,July
2019-07-31,"115,000","1,670","192,000,000",Florida,2019,July
2019-07-31,"81,000","1,765","143,000,000",Georgia,2019,July


### cast the columns to numeric

In [10]:
# disclosure_sentinels to handle
disc_sents_dct = {
    col: sum(["D" in str(cell) for cell in df[col].dropna().values])
    for col in df
                 }

if verbose:
    feedback = "Checking for count of Disclosure Nulls in each column"
    print(feedback + "\n" + "-"*len(feedback))
    print(disc_sents_dct)

Checking for count of Disclosure Nulls in each column
-----------------------------------------------------
{'Milk Cows': 0, 'Milk Per Cow (Lbs)': 0, 'Milk Production (Lbs)': 0, 'State': 4, 'Year': 0, 'Month': 0}


In [11]:
for col in df:
    if col not in {"State", "Year", "Month"}:
        df[col] = df[col].apply(
            lambda v: int("".join(v.split(","))) if type(v) == str else v
        )

In [12]:
df.head()

,Milk Cows,Milk Per Cow (Lbs),Milk Production (Lbs),State,Year,Month
Date,,,,,,
2019-07-31,195000,1940,378000000,Arizona,2019,July
2019-07-31,1725000,1965,3390000000,California,2019,July
2019-07-31,187000,2205,412000000,Colorado,2019,July
2019-07-31,115000,1670,192000000,Florida,2019,July
2019-07-31,81000,1765,143000000,Georgia,2019,July


### add fips codes to the data for the choropleth

In [13]:
# read the fips codes
state_fips_df = pd.read_csv(pathlib.Path("../geospatial-vis/states-fips.csv")).rename(columns = {'Name':'State'})
state_fips_df.head()

,State,Postal Code,FIPS
0,Alabama,AL,1
1,Alaska,AK,2
2,Arizona,AZ,4
3,Arkansas,AR,5
4,California,CA,6


In [14]:
# add the fips codes to the data
df = df.merge(state_fips_df, on = 'State')
df.head()

,Milk Cows,Milk Per Cow (Lbs),Milk Production (Lbs),State,Year,Month,Postal Code,FIPS
0,195000,1940,378000000,Arizona,2019,July,AZ,4
1,196000,1905,373000000,Arizona,2020,July,AZ,4
2,195000,1860,363000000,Arizona,2019,August,AZ,4
3,196000,1835,360000000,Arizona,2020,August,AZ,4
4,1725000,1965,3390000000,California,2019,July,CA,6


## Get a choropleth for each month for each year for each column

In [16]:
for year in df.Year.unique():
    for month in df.Month.unique():
        for col in df:
            # skip irrelevant columns
            if col not in {'Postal Code', 'State', 'FIPS', 'Year', "Month"}:
                # define the title
                ttl = f'{col} - 24 Selected States: {month} {year}'
                # define the choropleth
                fig = px.choropleth(
                    data_frame = df[(df.Year == year) & (df.Month == month)],
                    locations = 'Postal Code',
                    color = col,
                    locationmode = 'USA-states',
                    scope = "usa",
                    color_continuous_scale = "Viridis",
                    title = ttl
                )
                # show the figure
                if verbose:
                    fig.show()

                # export the figure in various formats
                for frmt in ['html', 'json', 'png']:
                    # prepare the path
                    outpath = str(
                        pathlib.Path(
                            re.sub(" ", "-", f"visualizations/choropleths/{frmt}/{col}-24-selected-states-{month}-{year}.{frmt}".lower())
                        )
                    )

                    if frmt == "png":
                        e = f"fig.write_image('{outpath}')"
                    else:
                        e = f"fig.write_{frmt}('{outpath}')"
                    eval(e)